In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [2]:
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/Forestaciones_uru')
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

In [3]:
roi_feature = ee.FeatureCollection(roi).first()
roi_geometry = roi_feature.geometry()
roi_coordinates = roi_geometry.coordinates().getInfo()


# Guardar las coordenadas en una variable
roi_coords = roi_coordinates
ee_roi =  ee.Geometry.Polygon(roi_coords) 

In [4]:
year = 2021

start = f'{year}-09-01'
end = f'{year}-12-31'

In [5]:
s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

cloud_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(roi) \
    .filterDate(start, end)

def mask_clouds(image):
    cloud_score = ee.Image(cloud_col.filterMetadata('system:index', 'equals', image.get('system:index')).first())
    mask = cloud_score.select('probability').lt(10)
    return image.updateMask(mask)

# Apply cloud masking and compute median
s2_masked = s2_col.map(mask_clouds)
s2_image = s2_masked.median().toFloat().clip(roi)


# Calculate indices (NDVI, MNDWI, NDBI, EVI, SAVI) for Sentinel-2
ndvi = s2_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
mndwi = s2_image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
ndbi = s2_image.normalizedDifference(['B11', 'B8']).rename('NDBI')

evi = s2_image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': s2_image.select('B8'),
        'RED': s2_image.select('B4'),
        'BLUE': s2_image.select('B2')
    }
).rename('EVI')

savi = s2_image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': s2_image.select('B8'),
        'RED': s2_image.select('B4')
    }
).rename('SAVI')


s2_image = s2_image.addBands([ndvi, mndwi, ndbi, evi, savi])

# Add Dynamic World label to the image
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dw_label = dw_col.select('label').median().clip(roi)
s2_image = s2_image.addBands(dw_label.rename('label'))

# Incorporate DEM and slope
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)
aspect = ee.Terrain.aspect(dem).rename('aspect').clip(roi)

def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

def create_filtered_collection(polarization):
    return ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
        .select(polarization) \
        .map(mask_edge)

img_vv = create_filtered_collection('VV')
img_vh = create_filtered_collection('VH')

# Compute medians for Sentinel-1
s1_vv = img_vv.median().rename('VV').clip(roi)
s1_vh = img_vh.median().rename('VH').clip(roi)

# Combine Sentinel-1 bands
s1_combined = ee.Image.cat([s1_vv, s1_vh])

# Calculate RVI, VV+VH, and VV-VH
rvi = s1_combined.expression(
    '4 * VH / (VV + VH)',
    {
        'VV': s1_combined.select('VV'),
        'VH': s1_combined.select('VH')
    }
).rename('RVI')

vv_plus_vh = s1_combined.expression(
    'VV + VH',
    {
        'VV': s1_combined.select('VV'),
        'VH': s1_combined.select('VH')
    }
).rename('VV_plus_VH')

vv_minus_vh = s1_combined.expression(
    'VV - VH',
    {
        'VV': s1_combined.select('VV'),
        'VH': s1_combined.select('VH')
    }
).rename('VV_minus_VH')

# Add bands to the image
s1_image = s1_combined.addBands([rvi, vv_plus_vh, vv_minus_vh])

# Combine all layers into the final image
image = s2_image.addBands([s1_image, dem.rename('elevation'), slope, aspect])

def calculate_texture_metrics(image, index_name, roi, scale_factor=100, size=3):
    
    index_norm = image.select(index_name).multiply(scale_factor).add(scale_factor).toInt32().clip(roi)
    
    glcm = index_norm.glcmTexture(size=size)
    contrast = glcm.select(f'{index_name}_contrast').rename(f'{index_name}_Contrast')
    correlation = glcm.select(f'{index_name}_corr').rename(f'{index_name}_Correlation')
    entropy = glcm.select(f'{index_name}_ent').rename(f'{index_name}_Entropy')
    inertia = glcm.select(f'{index_name}_inertia').rename(f'{index_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])



image = calculate_texture_metrics(image, 'NDVI', roi)
image = calculate_texture_metrics(image, 'SAVI', roi)

bands = ['B12', 'elevation', 'B8', 'B11', 'NDBI', 'slope', 'VH', 'MNDWI', 'B3', 'EVI', 'aspect', 'SAVI_Entropy']

image = image.select(bands)

In [24]:
# Cargar la colección Dynamic World
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dw_label = dw_col.select('label').median().clip(roi)

# Crear una máscara donde se mantengan solo los píxeles con clase 1 (bosques) o clase 6 (cultivos)
forest_and_crop_mask = dw_label.eq(1).Or(dw_label.eq(2))

# Aplicar la máscara a la imagen: todo lo que no sea clase 1 o clase 6 se convierte en NaN
image_masked = image.updateMask(forest_and_crop_mask)

# Donde no hay datos, asignar NaN
image = image_masked.unmask(ee.Image.constant(-9999)).clip(roi)


In [25]:
Map.centerObject(roi, 10)
Map.addLayer(image_masked, {}, 'VV', False)
Map

Map(bottom=311781.0, center=[-32.066057127217235, -58.303486187632885], controls=(WidgetControl(options=['posi…

In [22]:
feature_names =  ['B11', 'B12', 'B2', 'B3', 'B4', 'B8', 'EVI', 'MNDWI', 'NDBI', 'NDVI', 'RVI', 'SAVI', 'VH', 'VH_Contrast', 'VH_Correlation', 'VH_Entropy', 'VH_Inertia', 'VV', 'VV_Contrast', 'VV_Correlation', 'VV_Entropy', 'VV_Inertia', 'VV_minus_VH', 'VV_plus_VH', 'aspect', 'elevation', 'slope']

label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/Rf_AGB_trees-SN-CH"
asset_id


# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the ml.trees_to_classifier function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)


In [24]:
# feature_names =   ['B11', 'B12', 'B2','B3', 'B4', 'B8', 'CH', 'EVI', 'MNDWI', 'NDBI', 'NDVI', 'RVI', 'SAVI', 'VH', 'VH_Contrast', 'VH_Correlation', 'VH_Entropy', 'VH_Inertia', 'VV', 'VV_Contrast', 'VV_Correlation', 'VV_Entropy', 'VV_Inertia', 'VV_minus_VH', 'VV_plus_VH', 'aspect', 'elevation', 'label', 'slope']



# label = "agbd"

# user_id = 'users/facuboladerasgee'
# # specify asset id where to save trees
# # be sure to change  to your ee user name
# asset_id = user_id + "/Rf_CHM_pampa2"
# asset_id


# # Leer la colección de características exportada
# rf_fc = ee.FeatureCollection(asset_id)

# # Convertirlo en un clasificador
# another_classifier = ml.fc_to_classifier(rf_fc)

# # Aplicar una máscara para eliminar píxeles donde el valor es -9999
# valid_pixels = image.select(feature_names).mask(image.select(feature_names).neq(-9999))

# # Clasificar la imagen solo sobre los píxeles válidos
# classified = valid_pixels.classify(another_classifier)

# # O puedes asignar un valor específico a los píxeles no válidos después de la clasificación
# classified = classified.unmask(-9999)

In [7]:
feature_names =  ['B12', 'elevation', 'B8', 'B11', 'NDBI', 'slope', 'VH', 'MNDWI', 'B3', 'EVI', 'aspect', 'SAVI_Entropy']


label = "rh95"
user_id = 'users/facuboladerasgee'
def recombine_model_parts(user_id, base_asset_id, num_parts):
    parts = [ee.FeatureCollection(f"{base_asset_id}_part_{i}") for i in range(num_parts)]
    combined_fc = ee.FeatureCollection(parts[0])
    
    for part in parts[1:]:
        combined_fc = combined_fc.merge(part)
    
    return ml.fc_to_classifier(combined_fc)

# Ejemplo de uso:
num_parts = 2  # Ajusta este número según la cantidad de partes exportadas
base_asset_id = f"{user_id}/Rf_chm-costa-uru"
recombined_classifier = recombine_model_parts(user_id, base_asset_id, num_parts)

# Clasificar la imagen con el clasificador recombinado
classified = image.select(feature_names).classify(recombined_classifier)


In [8]:
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.centerObject(roi)


visPredictedBiomass = {
    'min': 0,
    'max': 25,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}

Map = geemap.Map(zoom=11)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

Map.addLayer(
    classified.clip(roi),
    visPredictedBiomass,
    "classification",)
Map.add_colorbar(visPredictedBiomass, label="Predicted Biomass")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [9]:
import time

# Parámetros de exportación
task = ee.batch.Export.image.toDrive(
    image=classified,
    description='chm-prueba',
    folder='EE_costa_uruguay_nuevo',
    region=ee_roi,
    scale=30,
    crs='EPSG:4326',  # CRS adecuado para Nepal
    
    
)
task.start()

print("Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.")

# Esperar a que la tarea de exportación se complete (opcional)
while task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

if task.status()['state'] == 'COMPLETED':
    print('Exportación completada con éxito.')
else:
    print('Error en la exportación:', task.status())

Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación completada con éxito.
